In [14]:
import seaborn as sns
import pandas as pd

# 1) Load + keep only columns we need
titanic = sns.load_dataset("titanic")
df = titanic[["sex","class","fare","survived","age"]].copy()

# 2) Drop rows with missing values in the fields we actually use
#    (drop rows, not columns)
df = df.dropna(subset=["sex","class","fare","survived","age"])

# 3) PIVOT: mean fare by sex × class  (wide format)
pivot = pd.pivot_table(
    df,
    values="fare",
    index="sex",
    columns="class",
    aggfunc="mean"
)
print("\nPIVOT (mean fare by sex × class):\n", pivot)

# 4) STACK: wide → long (columns 'class' folded into rows)
#    After stack, you get a Series with a name; reset_index to get a tidy DataFrame
stacked = pivot.stack().reset_index(name="avg_fare")
# columns now: ['sex', 'class', 'avg_fare']
print("\nSTACKED (long):\n", stacked.head())

# 5) UNSTACK: long → wide (rebuild the matrix)
#    Set a MultiIndex first, then unstack the 'class' level
unstacked = (stacked
             .set_index(["sex","class"])   # make the two keys an index
             .unstack("class"))            # spread 'class' back to columns
# After unstack, you get a DataFrame with a column MultiIndex ('avg_fare' on top).
# For a cleaner look, collapse it:
unstacked.columns = unstacked.columns.get_level_values(-1)  # keep only 'class' names
print("\nUNSTACKED (back to wide):\n", unstacked)

melted = df.melt(
    id_vars=["sex","class","survived"],  # keep these as identifiers
    value_vars=["fare","age"],           # turn these two columns into rows
    var_name="metric",
    value_name="value"
)
print("\nMELTED (tidy long fare/age):\n", melted.head())


compare = stacked.groupby(["sex","class"])["avg_fare"].mean()
print(compare)


PIVOT (mean fare by sex × class):
 class        First     Second      Third
sex                                     
female  107.946275  21.951070  15.875369
male     71.142781  21.113131  12.162695

STACKED (long):
       sex   class    avg_fare
0  female   First  107.946275
1  female  Second   21.951070
2  female   Third   15.875369
3    male   First   71.142781
4    male  Second   21.113131

UNSTACKED (back to wide):
 class        First     Second      Third
sex                                     
female  107.946275  21.951070  15.875369
male     71.142781  21.113131  12.162695

MELTED (tidy long fare/age):
       sex  class  survived metric    value
0    male  Third         0   fare   7.2500
1  female  First         1   fare  71.2833
2  female  Third         1   fare   7.9250
3  female  First         1   fare  53.1000
4    male  Third         0   fare   8.0500
sex     class 
female  First     107.946275
        Second     21.951070
        Third      15.875369
male    First      

/tmp/ipython-input-1362775218.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = pd.pivot_table(
/tmp/ipython-input-1362775218.py:47: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  compare = stacked.groupby(["sex","class"])["avg_fare"].mean()
